<a href="https://colab.research.google.com/github/mizeller/dh_project/blob/main/CalibrationAndMocap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Note

Before running this notebook make sure that your runtime type is 'Python 3 with GPU acceleration'. Go to Edit > Notebook settings > Hardware Accelerator > Select "GPU".

## Install OpenPose

This takes ~20min 


In [1]:
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git'
project_name = splitext(basename(git_repo_url))[0]

if not exists(project_name):
  !rm -rf openpose
  # see: https://github.com/CMU-Perceptual-Computing-Lab/openpose/issues/949
  print("install new CMake because of CUDA10")
  cmake_version = 'cmake-3.20.2-linux-x86_64.tar.gz'
  if not exists(cmake_version):
    !wget -q 'https://cmake.org/files/v3.20/{cmake_version}'
  !tar xfz {cmake_version} --strip-components=1 -C /usr/local

  print("clone openpose")
  !git clone -q --depth 1 $git_repo_url
  print("install system dependencies")
  !apt-get -qq install -y libatlas-base-dev libprotobuf-dev libleveldb-dev libsnappy-dev libhdf5-serial-dev protobuf-compiler libgflags-dev libgoogle-glog-dev liblmdb-dev opencl-headers ocl-icd-opencl-dev libviennacl-dev
  print("build openpose")
  # !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. && make -j`nproc`
  !cd openpose && rm -rf build || true && mkdir build && cd build && cmake .. -DUSE_CUDNN=OFF && make -j`nproc`


install new CMake because of CUDA10
clone openpose
install system dependencies
Selecting previously unselected package libgflags2.2.
(Reading database ... 122518 files and directories currently installed.)
Preparing to unpack .../00-libgflags2.2_2.2.2-1build1_amd64.deb ...
Unpacking libgflags2.2 (2.2.2-1build1) ...
Selecting previously unselected package libgflags-dev.
Preparing to unpack .../01-libgflags-dev_2.2.2-1build1_amd64.deb ...
Unpacking libgflags-dev (2.2.2-1build1) ...
Selecting previously unselected package libgoogle-glog0v5.
Preparing to unpack .../02-libgoogle-glog0v5_0.4.0-1build1_amd64.deb ...
Unpacking libgoogle-glog0v5 (0.4.0-1build1) ...
Selecting previously unselected package libgoogle-glog-dev.
Preparing to unpack .../03-libgoogle-glog-dev_0.4.0-1build1_amd64.deb ...
Unpacking libgoogle-glog-dev (0.4.0-1build1) ...
Selecting previously unselected package libleveldb1d:amd64.
Preparing to unpack .../04-libleveldb1d_1.22-3ubuntu2_amd64.deb ...
Unpacking libleveldb1d:a

## Install osmesa and pyrender for rendering

In [2]:
!sudo apt update > output.txt
!sudo wget https://github.com/mmatl/travis_debs/raw/master/xenial/mesa_18.3.3-0.deb > output.txt
!sudo dpkg -i ./mesa_18.3.3-0.deb || true > output.txt
!sudo apt install -f > output.txt



--2023-05-02 13:42:03--  https://github.com/mmatl/travis_debs/raw/master/xenial/mesa_18.3.3-0.deb
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/mmatl/travis_debs/master/xenial/mesa_18.3.3-0.deb [following]
--2023-05-02 13:42:03--  https://raw.githubusercontent.com/mmatl/travis_debs/master/xenial/mesa_18.3.3-0.deb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31776942 (30M) [application/octet-stream]
Saving to: ‘mesa_18.3.3-0.deb’

mesa_18.3.3-0.deb   100%[===================>]  30.30M  --.-KB/s    in 0.1s    

2023-05-02 13:42:04 (271 MB/s) - ‘mesa_18.3.3-0.deb’ saved [31776942/3177694

### Install pyopengl

In [3]:
!git clone https://github.com/mmatl/pyopengl.git  > output.txt
!pip install ./pyopengl  > output.txt

Cloning into 'pyopengl'...
remote: Enumerating objects: 26044, done.
remote: Total 26044 (delta 0), reused 0 (delta 0), pack-reused 26044
Receiving objects: 100% (26044/26044), 23.93 MiB | 25.99 MiB/s, done.
Resolving deltas: 100% (21690/21690), done.


In [4]:
!pip install ./pyopengl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./pyopengl
  Preparing metadata (setup.py) ... done
  Created wheel for PyOpenGL: filename=PyOpenGL-3.1.0-py3-none-any.whl size=2349651 sha256=be89f186fe13b9becfd37da95a65285d58ce5954f0799f858a06516598377b37
  Stored in directory: /tmp/pip-ephem-wheel-cache-eitfgpmj/wheels/27/c8/a3/78943947020a46f603940b3cd7f4da7ea2f4b95bc8cb53400b
Successfully built PyOpenGL
  Attempting uninstall: PyOpenGL
    Found existing installation: PyOpenGL 3.1.0
    Uninstalling PyOpenGL-3.1.0:
      Successfully uninstalled PyOpenGL-3.1.0


### Install PyRender

In [5]:
!git clone -q https://github.com/mmatl/pyrender.git > output.txt
!pip install pyrender > output.txt

## 2.Install EasyMocap

In [6]:
!git clone https://github.com/zju3dv/EasyMocap.git

Cloning into 'EasyMocap'...
remote: Enumerating objects: 1367, done.
remote: Counting objects: 100% (294/294), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 1367 (delta 172), reused 170 (delta 134), pack-reused 1073
Receiving objects: 100% (1367/1367), 65.78 MiB | 23.46 MiB/s, done.
Resolving deltas: 100% (612/612), done.


In [7]:
!pip install -r ./EasyMocap/requirements.txt > output.txt
!pip install chumpy > output.txt

In [8]:
%cd EasyMocap
!python setup.py develop --user
%cd ..

/content/EasyMocap
running develop
/usr/local/lib/python3.10/dist-packages/setuptools/command/develop.py:40: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/lib/python3.10/distutils/cmd.py:62: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See h

## download video data

In [9]:
!gdown "https://drive.google.com/uc?id=13Ls71Cdb2ErQfuZyDEepdXYjgc-I3laQ&export=download"
!unzip data.zip > output.txt
!mv ./data/smpl /content/EasyMocap/data/smplx/ && mv ./data/smplx /content/EasyMocap/data/smplx/

Downloading...
From: https://drive.google.com/uc?id=13Ls71Cdb2ErQfuZyDEepdXYjgc-I3laQ&export=download
To: /content/data.zip
100% 653M/653M [00:12<00:00, 51.7MB/s]
Archive:  data.zip
   creating: data/calibration/
   creating: data/calibration/extri_data/
   creating: data/calibration/extri_data/videos/
  inflating: data/calibration/extri_data/videos/1.mp4  
  inflating: data/calibration/extri_data/videos/2.mp4  
   creating: data/calibration/intri_data/
   creating: data/calibration/intri_data/videos/
  inflating: data/calibration/intri_data/videos/1.mp4  
  inflating: data/calibration/intri_data/videos/2.mp4  
   creating: data/motion/
   creating: data/motion/videos/
  inflating: data/motion/videos/1.mp4  
  inflating: data/motion/videos/2.mp4  
   creating: data/smpl/
  inflating: data/smpl/SMPL_NEUTRAL.pkl  
   creating: data/smplx/
  inflating: data/smplx/SMPLX_FEMALE.pkl  
  inflating: data/smplx/SMPLX_MALE.pkl  
  inflating: data/smplx/SMPLX_NEUTRAL.pkl  


# Camera Calibration

In [10]:
print('extra video, intrinsic data')

!python /content/EasyMocap/scripts/preprocess/extract_video.py /content/data/calibration/intri_data/ --no2d
print('extra video, extrinsic data')
!python /content/EasyMocap/scripts/preprocess/extract_video.py /content/data/calibration/extri_data/ --no2d



extra video, intrinsic data
1.mp4     : 100% 118/118 [00:45<00:00,  2.60it/s]
2.mp4     : 100% 131/131 [00:50<00:00,  2.62it/s]
cameras:  1 2
extra video, extrinsic data
1.mp4     : 100% 132/132 [00:53<00:00,  2.48it/s]
2.mp4     : 100% 132/132 [00:50<00:00,  2.61it/s]
cameras:  1 2


In [11]:
!mkdir /content/data/calibration/extri_data/output && mkdir /content/data/calibration/extri_data/output/calibration
!mkdir /content/data/calibration/intri_data/output && mkdir /content/data/calibration/intri_data/output/calibration

In [12]:
print('detect chessboard, intrinsic data')
!python /content/EasyMocap/apps/calibration/detect_chessboard.py /content/data/calibration/intri_data/ --out /content/data/calibration/intri_data/output/calibration --pattern 8,6 --grid 0.035

print('detect chessboard, extrinsic data')
!python /content/EasyMocap/apps/calibration/detect_chessboard.py /content/data/calibration/extri_data/ --out /content/data/calibration/extri_data/output/calibration --pattern 8,6 --grid 0.035


detect chessboard, intrinsic data
Create chessboard (8, 6)
create template chessboard: 100% 249/249 [00:00<00:00, 1165.96it/s]
- Load data from /content/data/calibration/intri_data/
- Try to find image names...
  -> find 249 images
- Try to find annot names...
  -> find 249 annots
detect chessboard, extrinsic data
Create chessboard (8, 6)
create template chessboard: 100% 264/264 [00:00<00:00, 1095.13it/s]
- Load data from /content/data/calibration/extri_data/
- Try to find image names...
  -> find 264 images
- Try to find annot names...
  -> find 264 annots
[Info] Cannot find chessboard in /content/data/calibration/extri_data/images/1/000011.jpg
[Info] Cannot find chessboard in /content/data/calibration/extri_data/images/1/000090.jpg
[Info] Cannot find chessboard in /content/data/calibration/extri_data/images/2/000001.jpg
[Info] Cannot find chessboard in /content/data/calibration/extri_data/images/2/000002.jpg
[Info] Cannot find chessboard in /content/data/calibration/extri_data/images

In [13]:
print('calibrate intrinsics')
!python3 /content/EasyMocap/apps/calibration/calib_intri.py /content/data/calibration/intri_data/


calibrate intrinsics
read: 100% 118/118 [00:00<00:00, 5471.60it/s]
[calibration] Load 118 images
>> Camera 1: 118 frames
-> [calibrate           ]:  17.4s
read: 100% 131/131 [00:00<00:00, 5861.90it/s]
[calibration] Load 131 images
>> Camera 2: 131 frames
-> [calibrate           ]:  23.1s


In [14]:
print('calibrate extrinsics')
!python3 /content/EasyMocap/apps/calibration/calib_extri.py /content/data/calibration/extri_data/ --intri /content/data/calibration/intri_data/output/intri.yml

calibrate extrinsics
1 center => [1.32572709 0.69533684 1.97347939], err = 0.303
2 center => [ 5.1630917  -3.39253661  6.46509411], err = 1.069


### Check calibration results

Stored in:
```
data/intri_data/output/1_used or 2_used
```
for some reason, this is different save path then what I get locally


If local, can also run annotation app to check and edit:

```
!python EasyMocap/apps/annotation/annot_calib.py /data/calibration/intri_data/ --mode chessboard --pattern 8,6 --annot chessboard
```




In [15]:
!mv /content/data/calibration/extri_data/extri.yml /content/data/motion
!mv /content/data/calibration/extri_data/intri.yml /content/data/motion


# Extract Openpose keypoints

Can take up to 15min

In [18]:
# can use highres argument to reduce resolution (if GPU resources are insufficient)
# from extract_video.py: add label --net_resolution -1x(int(16*((368*args.highres)//16)))
# for my machine locally, needed to use 1x256

272

In [19]:
!python /content/EasyMocap/scripts/preprocess/extract_video.py /content/data/motion --openpose /content/openpose/ #--highres 0

1.mp4     : 100% 542/542 [03:51<00:00,  2.34it/s]
2.mp4     : 100% 542/542 [03:16<00:00,  2.76it/s]
cameras:  1 2
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 90.458974 seconds.
1         : 100% 542/542 [02:02<00:00,  4.42it/s]
Starting OpenPose demo...
Configuring OpenPose...
Starting thread(s)...
Auto-detecting all available GPUs... Detected 1 GPU(s), using 1 of them starting at GPU 0.
OpenPose demo successfully finished. Total time: 80.372748 seconds.
2         : 100% 542/542 [01:52<00:00,  4.83it/s]


In [20]:
# if above line isn't working, run the following

# !gdown "https://drive.google.com/uc?id=1iSb3GzgRLpvBjeEwOffQPjPy9k1ZfRBQ&export=download"
# !unzip openpose_keypoints.zip -d ./data/motion > output.txt
# !python /content/EasyMocap/scripts/preprocess/extract_video.py /content/data/motion 

# Run Easy Mocap

Can take a long time (up to 30min)

In [21]:
%cd EasyMocap
!PYOPENGL_PLATFORM=osmesa python apps/demo/mv1p.py ../data/motion/ --out ../data/motion/output/smpl --vis_det --vis_repro --undis --sub_vis 1 2 --vis_smpl --model smpl --gender neutral --vis_smpl --start 235 --end 470
%cd ..

/content/EasyMocap

  Demo code for multiple views and one person:

    - Input : ../data/motion/ => 1, 2
    - Output: ../data/motion/output/smpl
    - Body  : smpl=>neutral, body25

triangulation: 100% 235/235 [05:26<00:00,  1.39s/it]
dump: 100% 235/235 [00:00<00:00, 6372.34it/s]
loading: 100% 235/235 [00:00<00:00, 3310.43it/s]
/content/EasyMocap/easymocap/pyfitting/lbfgs.py:264: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1485.)
  p.data.add_(step_size, update[offset:offset + numel].view_as(p.data))
-> [Optimize global RT  ]:   5.5s
-> [Optimize 3D Pose/235 frames]:  27.7s
-> [Optimize 2D Pose/235 frames]:  16.1s
render: 100% 235/235 [19:55<00:00,  5.09s/it]
/content


# Format Video

In [22]:
!ffmpeg -pattern_type glob -framerate 30 -i '/content/data/motion/output/smpl/smpl/*.jpg' -vf "scale=888:1000,pad=890:1000:(890-888)/2:0:black" -vcodec libx264 /content/data/motion/output/smpl/smpl.mp4

# !ffmpeg -framerate 30 -pattern_type glob -i '/content/data/motion/output/smpl/smpl/*.jpg' -c:v libx264 -pix_fmt yuv420p /content/data/motion/output/smpl/smpl.mp4


ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

In [23]:
from IPython.display import HTML
from base64 import b64encode

def video(path):
  mp4 = open(path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML('<video width=1024 controls loop> <source src="%s" type="video/mp4"></video>' % data_url)

video('/content/data/motion/output/smpl/smpl.mp4')

# Save Output

In [ ]:
# !zip -r /content/motion_output.zip /content/data/motion > zip_output.txt

In [30]:
!zip -r /content/motion_output.zip /content/data/motion/output > zip_output.txt

In [28]:
!zip -r /content/motion_openpose.zip /content/data/motion/openpose > zip_output.txt

In [31]:
from google.colab import files
files.download('/content/motion_output.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>